### set libraly

In [3]:
import pandas as pd
import os
import glob

### set path

In [2]:
input = "Input\\"
input_data = "Input\\data\\"
input_mapping = "Input\\mapping\\"
output = "Output\\"
dataprepare = "dataprepare\\"
#os.mkdir(input)
#os.mkdir(output)
#os.mkdir(input_data)
#os.mkdir(input_mapping)
#os.mkdir(dataprepare)

### prepare data

In [11]:
class RawData:
      def __init__(self, folder):
    
        self.folder = folder
        self.file = glob.glob(folder+"/*.csv")[-1]
    
        print(f"Start to read CSV file '{self.file[-26:]}' in folder '{self.folder}'")
        self.df = pd.read_csv(self.file,encoding='TIS-620',
                                    
                                    )
        print(f"Read CSV file '{self.file[-22:]}' done!","\n")
        
      def remove_white_space(self):
            print("Start remove_white_space function")
            for col in self.df.columns.tolist():
              if self.df[col].dtypes == object:
                self.df[col] = self.df[col].str.strip()
              print("Remove_white_space done!","\n")
      def save_csv(self , fname):
              self.df.to_csv(fname,encoding='TIS-620')
              print(f"save file {fname} done" , "\n")
     
    

In [12]:
test = RawData(input_data)
test.remove_white_space()
test.save_csv(input_data+'test.csv')

Start to read CSV file 'Input\data\test.csv' in folder 'Input\data\'
Read CSV file 'Input\data\test.csv' done! 

Start remove_white_space function
Remove_white_space done! 

Remove_white_space done! 

Remove_white_space done! 

Remove_white_space done! 

Remove_white_space done! 

Remove_white_space done! 

Remove_white_space done! 

save file Input\data\test.csv done 



In [5]:
a = pd.read_csv(input_data+'CRR2563.csv',encoding='UTF-8',
                                    low_memory=False,skipinitialspace=True,
                                    )

In [6]:
a

,บุคคล/นิติบุคคลที่ถูกประเมิน,เลขที่ CIF,วันที่ประเมิน,Grade,ประเภท,เงื่อนไข
0,บริษัท ป่าตองเพลส รีสอร์ท คลับ,550004812,12/2/2563,C+,M & L Services,เปลี่ยนแปลงเงื่อนไข
1,บริษัท ศรีวิชัยเวชวิวัฒน์ จำกัด(มหาชน),750760423,14/1/2563,BBB,M & L Services,Credit Review
2,บมจ.ฟินันซ่า,590002150,27/8/2563,C+,M & L Services,Credit Review
3,บริษัท บุญบารมีเมตตา พร็อพเพอร์ตี้ จำกัด,540004526,30/4/2563,C+,M & L Services,เปลี่ยนแปลงเงื่อนไข
4,นางเอมพิกา จิตต์เสนา,730016502,4/7/2563,CC,M & L Services,เปลี่ยนแปลงเงื่อนไข
...,...,...,...,...,...,...
1340,พาอีซะ สุขพร,580003268,12/2/2563,sB+,S industries,พิจารณาสินเชื่อ
1341,ศิริพร และ ยุทธชัย ช่วยชูหนู,750843769,21/12/2563,sCCC,S industries,NaN
1342,ศิริพร และ ยุทธชัย ช่วยชูหนู,750843766,21/12/2563,sCCC,S industries,NaN
1343,สมัชชา ไซห์ฮามิส,680016495,1/12/2563,sCC,S industries,Credit Review
